In [2]:
%matplotlib inline
import subprocess
import netCDF4
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import *
import pytz
import itertools

NetCDF Windows library is buggy! Download files manually from:
http://opendap.tudelft.nl/thredds/data2/zandmotor/morphology/JETSKI/gridded/catalog.html

In [3]:
# based on TUDelft repository
urls = ([
    '../data/Jetski/jetskikb118_3736.nc',
    '../data/Jetski/jetskikb118_3938.nc',
    '../data/Jetski/jetskikb117_3938.nc'
])

In [4]:
grids = []
for url in urls:
    ds = netCDF4.Dataset(url)
    times = netCDF4.num2date(ds.variables['time'][:], ds.variables['time'].units)
    local = pytz.timezone("Europe/Amsterdam")
    times = [local.localize(t, is_dst=None).astimezone(pytz.utc) for t in times]
    arrs = []
    z = ds.variables['z'][:]
    x = ds.variables['x'][:]
    y = ds.variables['y'][:]
    grids.append({
        "url": url,
        "x": x,
        "y": y,
        "z": z,
        "times": times
    })
    ds.close()


In [5]:
count = len(list(itertools.chain.from_iterable([g['times'] for g in grids])))
count

97

In [6]:
ee_collection_path = 'users/gena/eo-bathymetry/sandengine_jetski'

In [11]:
def run(cmd):
    print(cmd)
    subprocess.call(cmd)

In [15]:
i = 0
for g in grids:
    ncols = len(g['x'])
    nrows = len(g['y'])
    cellsize = g['x'][1] - g['x'][0]
    xllcorner = np.min(g['x'])
    yllcorner = np.min(g['y'])
    nodata_value = -9999

    for t in g['times']:
        print('Processing time: {0}'.format(t))
        z = g['z']
        filename = 'bathymetry_jetski_' + str(i).rjust(3, '0')
        filepath = '../output/' + filename
        filepath_asc = filepath + '.asc'
        filepath_tif = filepath + '.tif'
        filename_tif = filename + '.tif'

        z = z[i]

        with open(filepath_asc, 'w') as f:
            f.write('ncols {0}\n'.format(ncols))
            f.write('nrows {0}\n'.format(nrows))
            f.write('cellsize {0}\n'.format(cellsize))
            f.write('xllcorner {0}\n'.format(xllcorner))
            f.write('yllcorner {0}\n'.format(yllcorner))
            f.write('nodata_value {0}\n'.format(nodata_value))
            for row in np.flip(range(nrows), 0):
                s = ' '.join([str(v) for v in z[row,]]).replace('--', str(nodata_value))
                f.write(s)
                f.write('\n')

        cmd = 'gdal_translate -ot Float32 -a_srs EPSG:28992 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=6 -of GTiff {0} {1}'\
            .format(filepath_asc, filepath_tif)
        run(cmd)
        
        filepath_gs = 'gs://hydro-earth/sandengine/' + filename_tif
        
        gsutil = 'D:/src/google-cloud-sdk/bin/gsutil.cmd' # relative path is not defined on Windows
        cmd = gsutil + ' cp {0} {1}'\
            .format(filepath_tif, filepath_gs)
        run(cmd)
        
        filepath_ee = ee_collection_path + '/' + filename        
        cmd = 'earthengine upload image --wait --asset_id={0} --nodata_value={1} {2}'\
            .format(filepath_ee, nodata_value, filepath_gs)        
        run(cmd)
        
        time_start = int(grids[0]['times'][0].timestamp() * 1000)
        cmd = 'earthengine asset set --time_start {0} {1}'\
            .format(time_start, filepath_ee)
        run(cmd)

        cmd = 'earthengine acl set public {0}'\
            .format(filepath_ee)
        run(cmd)

    i = i + 1

Processing time: 2011-08-02 22:00:00+00:00
gdal_translate -ot Float32 -a_srs EPSG:28992 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=6 -of GTiff ../output/bathymetry_jetski_000.asc ../output/bathymetry_jetski_000.tif
D:/src/google-cloud-sdk/bin/gsutil.cmd cp ../output/bathymetry_jetski_000.tif gs://hydro-earth/sandengine/bathymetry_jetski_000.tif
earthengine upload image --wait --asset_id=users/gena/eo-bathymetry/sandengine_jetski/bathymetry_jetski_000 --nodata_value=-9999 gs://hydro-earth/sandengine/bathymetry_jetski_000.tif
earthengine asset set --time_start 1312322400000 users/gena/eo-bathymetry/sandengine_jetski/bathymetry_jetski_000
Processing time: 2011-09-03 22:00:00+00:00
gdal_translate -ot Float32 -a_srs EPSG:28992 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=6 -of GTiff ../output/bathymetry_jetski_000.asc ../output/bathymetry_jetski_000.tif
D:/src/google-cloud-sdk/bin/gsutil.cmd cp ../output/bathymetry_jetski_000.tif gs://hydro-earth/sandengine/bathymetry_jetski_000.tif

KeyboardInterrupt: 